# LeRobot v0.4.0

## 概要

[LeRobot v0.4.0][1]の新機能:

[1]: https://github.com/huggingface/lerobot/releases/tag/v0.4.0

- Dataset v3の導入
    - 1エピソード1ファイルから、複数のエピソードを単一のParquet/MP4ファイルに集約する形式に変更
    - エピソードの参照にリレーショナルメタデータを使用
    - StreamingLeRobotDatasetによりデータセットのストリームでのダウンロードが可能になった
    - v2.1データセットをv3に変換するスクリプトを追加 
- 新しいポリシーの追加
    - NVIDIA GR00TN 1.5を追加
    - LiberoとMetaWorldのシミュレーション環境を追加
    - Reachy 2ロボットとIntel XPUバックエンドへの対応を追加
- パフォーマンスの向上
    - accelerateライブラリを用いたマルチGPUトレーニングに対応
    - データ拡張機能にアフィン変換を追加
    - APIの拡張（build_inference_frame・make_robot_action）
- コードベースの大幅なリファクタリング
    - RL・非同期処理関連の実装がトップレベルに移動
    - mypyによる型チェックを導入
    - 古いコンポーネントを非推奨化

## 検証設定

In [ ]:
%load_ext autoreload
%autoreload 2

## インストール

[Install LeRobot][1]

[1]: https://huggingface.co/docs/lerobot/en/installation#install-lerobot-

In [ ]:
import torch
assert torch.cuda.is_available(), "CUDA is not available"
torch.__version__

In [ ]:
!conda install ffmpeg=7.1.1 -c conda-forge -y

In [ ]:
# https://pypi.org/project/torchcodec/
%pip install -U "torchcodec<0.3"

In [ ]:
import torchcodec
torchcodec.__version__

In [ ]:
!python --version # 3.10
!ffmpeg -version # 7.X

In [ ]:
import os

if not os.path.exists("lerobot"):
    !git clone https://github.com/huggingface/lerobot.git

%cd lerobot
%pip install -qe . # LeRobotの依存関係をインストール
%pip show lerobot # 0.4.1

## Dataset v3

### エピソードベースからファイルベースへ

Dataset v2.1は、ファイルシステムの制限により数百万ものエピソードに対応できなかった

Dataset v3では、複数のエピソードをまとめたファイルベースに変更され、より大きなファイルで管理するようになった:

![](image/dataset.png)

### データセットの構成

- テーブルデータ
    - 状態・アクション・タイムスタンプ
    - Apache Parquet形式
- 視覚データ
    - カメラフレームは連結されMP4にエンコード
    - 同一エピソードでグループ化
    - シャーディングはカメラごとに行う
- メタデータ
    - フレームレート・統計量・共有Parquet/MP4ファイルへのエピソードの開始終了オフセット 
    - JSON・Parquet形式

### v2.1からv3.0への移行

コンバーターの動作:

- Parquetファイルを集約
- MP4ファイルを集約
- meta/episodes/*をエピソードごとの長さ、タスク、バイト・フレームオフセットで更新する

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
assert HF_TOKEN is not None, "Please set HF_TOKEN in the .env file"

# リポジトリを指定
repo_id = "EngineerCafeJP/record-test-2025-10-07-21-26-29"

# convert_dataset_v21_to_v30.pyを実行してデータセットを変換し、上書きアップロード
!export HF_TOKEN=$HF_TOKEN && \
    python -m lerobot.datasets.v30.convert_dataset_v21_to_v30 --repo-id=$repo_id

### データセットのロード

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

repo_id = "EngineerCafeJP/record-test-2025-10-07-21-26-29"

# TODO: torchcodecだとVideoDecoderの読み込みに失敗する
dataset = LeRobotDataset(repo_id, video_backend="pyav")

sample = dataset[100]
sample.keys()

In [ ]:
# デルタタイムスタンプを使って時系列ウィンドウを取得

# 現在のフレームから-0.2秒、-0.1秒、0.0秒の3フレームを取得
# TODO: v3.0移行でキー名が壊れている
delta_timestamps = {
    "observation.images.images.top": [-0.2, -0.1, 0.0]
}

dataset = LeRobotDataset(repo_id, delta_timestamps=delta_timestamps, video_backend="pyav")

# Accessing an index now returns a stack for the specified key(s)
sample = dataset[100]
sample["observation.images.images.top"].shape  # torch.Size([3, 3, H, W])

In [ ]:
# 訓練時はDataLoaderでラップする

batch_size = 16
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

device = "cuda" if torch.cuda.is_available() else "cpu"
for batch in data_loader:
    observations = batch["observation.state"].to(device)
    actions = batch["action"].to(device)
    images = batch["observation.images.images.top"].to(device)
    # model.forward(batch)
    break

### ストリーミング

In [ ]:
from lerobot.datasets.streaming_dataset import StreamingLeRobotDataset

repo_id = "EngineerCafeJP/record-test-2025-10-07-21-26-29"
# torchcodecしか対応していない
dataset = StreamingLeRobotDataset(repo_id)

### ディレクトリレイアウト

[EngineerCafeJP/record-test-2025-10-07-21-26-29][1]

[1]: https://huggingface.co/datasets/EngineerCafeJP/record-test-2025-10-07-21-26-29/tree/main

- meta/info.json 
    - スキーマ
    - FPS
    - コードベースのバージョン
    - データ・動画シャードを見つけるためのパステンプレート
- meta/stats.json
    - 正規化に使用するグローバルな特徴量の統計
- meta/tasks.jsonl
    - 整数IDとタスクのマッピング
- meta/episodes/
    - エピソードごとの記録（長さ、タスク、オフセット）
    - チャンク化されたParquet形式
- data/
    - フレームごとのParquetシャード
    - 各ファイルに多くのエピソードが含まれる
- videos/
    - カメラごとのMP4シャード
    - 各ファイルに多くのエピソードが含まれる

### 訓練時の画像のデータ拡張

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset
from lerobot.datasets.transforms import ImageTransforms, ImageTransformsConfig, ImageTransformConfig
import matplotlib.pyplot as plt

# オプション1：デフォルトの変換設定を使用（デフォルトでは無効）
transforms_config = ImageTransformsConfig(
    enable=True,  # 変換を有効化
    max_num_transforms=3,  # 1フレームあたり最大3つの変換を適用
    random_order=False,  # 標準の順序で適用
)
transforms = ImageTransforms(transforms_config)

repo_id = "EngineerCafeJP/record-test-2025-10-07-21-26-29"

dataset = LeRobotDataset(
    repo_id=repo_id,
    image_transforms=transforms,
    video_backend="pyav",
)

sample = dataset[100]
image = sample["observation.images.images.top"]
plt.imshow(image.permute(1, 2, 0))
plt.axis("off")

In [ ]:
# オプション2：カスタムの変換設定を作成
custom_transforms_config = ImageTransformsConfig(
    enable=True,
    max_num_transforms=2,
    random_order=True,
    tfs={
        "brightness": ImageTransformConfig(
            weight=1.0,
            type="ColorJitter",
            kwargs={"brightness": (0.7, 1.3)}  # 明るさの範囲を調整
        ),
        "contrast": ImageTransformConfig(
            weight=2.0,  # 重みが大きいほど選択されやすい
            type="ColorJitter",
            kwargs={"contrast": (0.8, 1.2)}
        ),
        "sharpness": ImageTransformConfig(
            weight=0.5,  # 重みが小さいほど選択されにくい
            type="SharpnessJitter",
            kwargs={"sharpness": (0.3, 2.0)}
        ),
    }
)

repo_id = "EngineerCafeJP/record-test-2025-10-07-21-26-29"

dataset = LeRobotDataset(
    repo_id=repo_id,
    image_transforms=ImageTransforms(custom_transforms_config),
    video_backend="pyav",
)

sample = dataset[100]
image = sample["observation.images.images.top"]
plt.imshow(image.permute(1, 2, 0))
plt.axis("off")
plt.show()

In [ ]:
# オプション3：純粋なtorchvisionの変換を使用
from torchvision.transforms import v2

torchvision_transforms = v2.Compose([
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    v2.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
])

repo_id = "EngineerCafeJP/record-test-2025-10-07-21-26-29"

dataset = LeRobotDataset(
    repo_id=repo_id,
    image_transforms=torchvision_transforms
)

plt.imshow(image.permute(1, 2, 0))
plt.axis("off")
plt.show()

### ベストプラクティス

- 控えめに始める: 小さな範囲（明るさ0.9-1.1など）からはじめ徐々に増やす
- テストする: 変換後の画像を確認する
- トレーニングを監視する: 順調に学習できているかの経過を観察する
- ドメインに合わせる: 照明条件が変わる場合は、明るさ・コントラストの変換を使用する
- 賢く組み合わせる: 同時に多くの変換を適用するとトレーニングが不安定になる